In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ID in test set (corresponds to a shop_id and item_id, predicted number of items sold
sample_submission = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
# items: item_name, item_id, item_category_id
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv") 
# item_categories: item_category_name, item_category_id
item_categories = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
# sales_train: date, date_block_num, shop_id, item_id, item_price, item_cnt_day**
sales_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
# shops: shop_name, shop_id
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

In [ ]:
print(items.info())
print('Items : \n\t'+'\n\t'.join(list(items)))
print('ItemCatagories : \n\t'+'\n\t'.join(list(item_categories.columns.values)))
print('Shops : \n\t'+'\n\t'.join(shops.columns.tolist()))
print('SalesTrain : \n\t'+'\n\t'.join(sales_train.columns.tolist()))

print('Test : \n\t'+'\n\t'.join(list(test)))
print('OutPut : \n\t'+'\n\t'.join(list(sample_submission)))

sales_train.info()

In [ ]:
print("Items")
print(items.head(2))
print("\nItem Catagerios")
print(item_categories.tail(2))
print("\nShops")
print(shops.sample(n=2))
print("\nTraining Data Set")
print(sales_train.sample(n=3,random_state=1))
print("\nTest Data Set")
print(test.sample(n=3,random_state=1))

In [ ]:
from datetime import datetime
sales_train['year'] = pd.to_datetime(sales_train['date']).dt.strftime('%Y')
sales_train['month'] = sales_train.date.apply(lambda x: datetime.strptime(x,'%d.%m.%Y').strftime('%m')) 

sales_train.head(2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#grafik çıktılarınızın görünmesini ve not defterinde saklanmasını sağlayacaktır.
%matplotlib inline 

grouped = pd.DataFrame(sales_train.groupby(['year','month'])['item_cnt_day'].sum().reset_index())
sns.pointplot(x='month', y='item_cnt_day', hue='year', data=grouped)

In [ ]:
#Price
grouped_price = pd.DataFrame(sales_train.groupby(['year','month'])['item_price'].mean().reset_index())
sns.pointplot(x='month', y='item_price', hue='year', data=grouped_price)

In [ ]:
ts=sales_train.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,8))
plt.title('Total Sales of the whole time period')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts);

In [ ]:
sns.jointplot(x="item_cnt_day", y="item_price", data=sales_train, height=8)
plt.show()

In [ ]:
sales_train.item_cnt_day.hist(bins=100)
sales_train.item_cnt_day.describe()

In [ ]:
print('item_price 0ları temizlemeden önce veri kümesi boyutu:', sales_train.shape)
sales_train = sales_train.query('item_price > 0')
print('item_price 0ları temizledikten sonra veri kümesi boyutu:', sales_train.shape)

In [ ]:
print('Filtreden önceki veri kümesi boyutu:', sales_train.shape)
# Yalnızca test setinde bulunan shops.
sales_train = sales_train[sales_train['shop_id'].isin(test['shop_id'].unique())]
# Yalnızca test setinde bulunan items.
sales_train = sales_train[sales_train['item_id'].isin(test['item_id'].unique())]
print('Filtreden sonraki veri kümesi boyutu:', sales_train.shape)

In [ ]:
print('Aykırı değerleri kaldırmadan önce veri kümesi boyutu:', sales_train.shape)
sales_train = sales_train.query('item_cnt_day >= 0 and item_cnt_day <= 125 and item_price < 75000')
print('Aykırı değerleri kaldırdıktan sonra veri kümesi boyutu:', sales_train.shape)

In [ ]:
#Grafik temizlendikten sonra
sns.jointplot(x="item_cnt_day", y="item_price", data=sales_train, height=8)
plt.show()

cleaned = pd.DataFrame(sales_train.groupby(['year','month'])['item_cnt_day'].sum().reset_index())
sns.pointplot(x='month', y='item_cnt_day', hue='year', data=cleaned)

In [ ]:
# Aylık satış seviyesi toplamı
monthly_sales=sales_train.groupby(["date_block_num","shop_id","item_id"])[
    "date_block_num","date","item_price","item_cnt_day"].agg({"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

monthly_sales.head(5)

In [ ]:
sales_data_flat = monthly_sales.item_cnt_day.apply(list).reset_index()
#Yalnızca geçerli test verilerini saklayın
sales_data_flat = pd.merge(test,sales_data_flat,on = ['item_id','shop_id'],how = 'left')
#fill na with 0
sales_data_flat.fillna(0,inplace = True)
sales_data_flat.drop(['shop_id','item_id'],inplace = True, axis = 1)
sales_data_flat.head(20)

In [ ]:
# Pivot tablo oluşturacağız.
# Satır = her shop+item kodu
# Sütunlar zaman aşımı dizisi olacak
pivoted_sales = sales_data_flat.pivot_table(index='ID', columns='date_block_num',fill_value = 0,aggfunc='sum' )
pivoted_sales.head(20)

Bölünmüş eğitim, doğrulama ve test veri kümesi


In [ ]:
X_train = np.expand_dims(pivoted_sales.values[:,:-1],axis = 2)
y_train = pivoted_sales.values[:,-1:]
X_test = np.expand_dims(pivoted_sales.values[:,1:],axis = 2)
print(X_train.shape,y_train.shape,X_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model

# tanımlayıcı satış modelimiz
sales_model = Sequential()
sales_model.add(LSTM(units = 64,input_shape = (33,1)))
sales_model.add(Dropout(0.5))
sales_model.add(Dense(1))

sales_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
sales_model.summary()

In [ ]:
sales_model.fit(X_train,y_train,batch_size = 4096,epochs = 10)

In [ ]:
submission_output = sales_model.predict(X_test)
# gerekli sütunlarla veri çerçevesi oluşturma
sample_submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_output.ravel()})
# dataframe'den csv dosyası oluşturma
sample_submission.to_csv('submission.csv',index = False)
sample_submission.head()